# Customer Segmentation Retention Analysis

Author -  Siddharth Patondikar

### Importing Data

In [92]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl

In [93]:
#Importing data
df = pd.read_csv("../data/raw/online_retail.csv")

print(df.shape)

(1067371, 8)


In [94]:
print(df.head(10).to_string())

  Invoice StockCode                          Description  Quantity          InvoiceDate  Price  Customer ID         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12  2009-12-01 07:45:00   6.95      13085.0  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS        12  2009-12-01 07:45:00   6.75      13085.0  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS        12  2009-12-01 07:45:00   6.75      13085.0  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48  2009-12-01 07:45:00   2.10      13085.0  United Kingdom
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24  2009-12-01 07:45:00   1.25      13085.0  United Kingdom
5  489434     22064           PINK DOUGHNUT TRINKET POT         24  2009-12-01 07:45:00   1.65      13085.0  United Kingdom
6  489434     21871                  SAVE THE PLANET MUG        24  2009-12-01 07:45:00   1.25      13085.0  United Kingdom
7  48943

In [95]:
print(df.dtypes.to_string())

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object


In [96]:
print(df.isnull().sum().to_string())

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0


In [97]:
dupes = df.duplicated()
print(f"Duplicate rows: {dupes.sum():,} ({dupes.sum()/len(df)*100:.2f}%)")

Duplicate rows: 34,335 (3.22%)


### Data Wrangling

In [98]:
#Dropping Duplicate Rows
df = df.drop_duplicates(keep='first')

dupes = df.duplicated()
print(f"Duplicate rows: {df.duplicated().sum():,} ({dupes.sum()/len(df)*100:.2f}%)")

Duplicate rows: 0 (0.00%)


In [99]:
print(df.head().to_string())

  Invoice StockCode                          Description  Quantity          InvoiceDate  Price  Customer ID         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12  2009-12-01 07:45:00   6.95      13085.0  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS        12  2009-12-01 07:45:00   6.75      13085.0  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS        12  2009-12-01 07:45:00   6.75      13085.0  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48  2009-12-01 07:45:00   2.10      13085.0  United Kingdom
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24  2009-12-01 07:45:00   1.25      13085.0  United Kingdom


In [100]:
# Identifying cancelled orders (Invoice starts with 'C')
cancelled = df[df['Invoice'].astype(str).str.startswith('C')]
print(f"Cancelled transactions: {len(cancelled):,} ({len(cancelled)/len(df)*100:.2f}%)")
print(f"Cancelled invoices: {cancelled['Invoice'].nunique():,}")
print(f"\nSample cancelled orders:")
print(cancelled.head().to_string())

Cancelled transactions: 19,104 (1.85%)
Cancelled invoices: 8,292

Sample cancelled orders:
     Invoice StockCode                    Description  Quantity          InvoiceDate  Price  Customer ID    Country
178  C489449     22087       PAPER BUNTING WHITE LACE       -12  2009-12-01 10:33:00   2.95      16321.0  Australia
179  C489449    85206A   CREAM FELT EASTER EGG BASKET        -6  2009-12-01 10:33:00   1.65      16321.0  Australia
180  C489449     21895  POTTING SHED SOW 'N' GROW SET        -4  2009-12-01 10:33:00   4.25      16321.0  Australia
181  C489449     21896             POTTING SHED TWINE        -6  2009-12-01 10:33:00   2.10      16321.0  Australia
182  C489449     22083     PAPER CHAIN KIT RETRO SPOT       -12  2009-12-01 10:33:00   2.95      16321.0  Australia


In [101]:
# Identifying rows with negative or zero Quantity and Price
neg_qty = df[df['Quantity'] <= 0]
neg_price = df[df['Price'] <= 0]
zero_price = df[df['Price'] == 0]

print(f"Negative/zero Quantity rows: {len(neg_qty):,} ({len(neg_qty)/len(df)*100:.2f}%)")
print(f"Negative Price rows: {len(df[df['Price'] < 0]):,}")
print(f"Zero Price rows: {len(zero_price):,}")
print(f"\nOverlap: cancelled orders with negative quantity: {len(cancelled[cancelled['Quantity'] < 0]):,}")

Negative/zero Quantity rows: 22,496 (2.18%)
Negative Price rows: 5
Zero Price rows: 6,014

Overlap: cancelled orders with negative quantity: 19,103


So from above:
- Cancelled Orders : 19,104
- Negative Qty :  19,103

There is one cancelled order with positive qty

In [102]:
#Identifying positive qty with cancelled order
anomaly = df[(df['Invoice'].astype(str).str.startswith('C')) & (df['Quantity'] >= 0)]

print("The Anomaly Row:")
print(anomaly.to_string())

The Anomaly Row:
       Invoice StockCode Description  Quantity          InvoiceDate   Price  Customer ID         Country
76799  C496350         M      Manual         1  2010-02-01 08:24:00  373.57          NaN  United Kingdom


The customer ID here is null which will be dropped in data cleaning part

In [103]:
#Checking customer ID null values
df_temp = df[df['Customer ID'].isna()]
print(f"Total null rows in Customer ID: {len(df_temp)}")
print(f"Percentage of null: {round((len(df_temp)/len(df))*100,2)}%")
print(df_temp["Customer ID"].head().to_string())

Total null rows in Customer ID: 235151
Percentage of null: 22.76%
263   NaN
283   NaN
284   NaN
470   NaN
577   NaN


Customer ID is a float here, but should ideally not have any decimal values, checking that

In [104]:
# Returns True if any customeriD value has a decimal > 0
has_decimals = (df['Customer ID'] % 1 > 0).any()

print(f"Are there actual decimals? {has_decimals}")

Are there actual decimals? False


Hence, Customer ID should be treated as a categorical variable instead of a numerical one

In [ ]:
print(df[df['Price']<0].head().to_string())

        Invoice StockCode      Description  Quantity         InvoiceDate     Price  Customer ID         Country
179403  A506401         B  Adjust bad debt         1 2010-04-29 13:36:00 -53594.36          NaN  United Kingdom
276274  A516228         B  Adjust bad debt         1 2010-07-19 11:24:00 -44031.79          NaN  United Kingdom
403472  A528059         B  Adjust bad debt         1 2010-10-20 12:04:00 -38925.87          NaN  United Kingdom
825444  A563186         B  Adjust bad debt         1 2011-08-12 14:51:00 -11062.06          NaN  United Kingdom
825445  A563187         B  Adjust bad debt         1 2011-08-12 14:52:00 -11062.06          NaN  United Kingdom


Even here the negative prices belong to null customer IDs

Next Checking the null descriptions

In [124]:
print(df[df["Description"].isna()].head().to_string())

     Invoice StockCode Description  Quantity         InvoiceDate  Price  Customer ID         Country
470   489521     21646         NaN       -50 2009-12-01 11:44:00    0.0          NaN  United Kingdom
3114  489655     20683         NaN       -44 2009-12-01 17:26:00    0.0          NaN  United Kingdom
3161  489659     21350         NaN       230 2009-12-01 17:39:00    0.0          NaN  United Kingdom
3731  489781     84292         NaN        17 2009-12-02 11:45:00    0.0          NaN  United Kingdom
4296  489806     18010         NaN      -770 2009-12-02 12:42:00    0.0          NaN  United Kingdom


Checking if Customer ID is null for all null descriptions

In [125]:
desc_null = df['Description'].isna().sum()
both_null = df[df['Description'].isna() & df['Customer ID'].isna()].shape[0]

print(f"Number of rows with null description: {desc_null}")
print(f"Number of rows with both null description and customer id: {both_null}")

Number of rows with null description: 4275
Number of rows with both null description and customer id: 4275


So all null descriptions also have null customer IDs

#### Data Cleaning

In [126]:
print(f"Starting rows: {len(df):,}\n")

Starting rows: 1,033,036



In [127]:
#Dropping null Customer IDs
df_clean = df.dropna(subset=["Customer ID"]).copy()
print(f"After dropping null Customer ID: {len(df_clean):,}")

After dropping null Customer ID: 797,885


In [128]:
# Converting Data Types
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

print(f"Date range: {df_clean['InvoiceDate'].min()} to {df_clean['InvoiceDate'].max()}")
print(f"Time span: {(df_clean['InvoiceDate'].max() - df_clean['InvoiceDate'].min()).days} days")

df_clean['Customer ID'] = df_clean['Customer ID'].astype('Int64')
print("\n"+df_clean.head().to_string())

Date range: 2009-12-01 07:45:00 to 2011-12-09 12:50:00
Time span: 738 days

  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12 2009-12-01 07:45:00   6.95        13085  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75        13085  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75        13085  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48 2009-12-01 07:45:00   2.10        13085  United Kingdom
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24 2009-12-01 07:45:00   1.25        13085  United Kingdom


In [130]:
# Creating New cols

# Creating a flag for cancelled orders
df_clean["IsCancelled"] = df_clean['Invoice'].astype(str).str.startswith('C')
print(f"Cancelled transactions: {df_clean['IsCancelled'].sum():,} ({df_clean['IsCancelled'].mean()*100:.2f}%)")
print(f"Normal transactions:    {(~df_clean['IsCancelled']).sum():,}")

Cancelled transactions: 18,390 (2.30%)
Normal transactions:    779,495


In [131]:
# Creating total amount col
df_clean['TotalAmount'] = df_clean["Quantity"]*df_clean["Price"]

print(f"TotalAmount stats (all transactions):")
print(df_clean['TotalAmount'].describe().to_string())

print(f"\nTotalAmount for cancelled orders:")
print(df_clean[df_clean['IsCancelled']]['TotalAmount'].describe().to_string())

print(f"\nTotalAmount for normal orders:")
print(df_clean[~df_clean['IsCancelled']]['TotalAmount'].describe().to_string())


TotalAmount stats (all transactions):
count    797885.000000
mean         20.416465
std         313.518824
min     -168469.600000
25%           4.350000
50%          11.700000
75%          19.500000
max      168469.600000

TotalAmount for cancelled orders:
count     18390.000000
mean        -58.989287
std        1437.408776
min     -168469.600000
25%         -17.700000
50%          -8.750000
75%          -3.750000
max          -0.120000

TotalAmount for normal orders:
count    779495.000000
mean         22.289821
std         227.416962
min           0.000000
25%           4.950000
50%          12.480000
75%          19.800000
max      168469.600000


In [136]:
# Check remaining zero-price rows
zero_price = df_clean[df_clean['Price'] == 0]
print(f"Zero price rows remaining: {len(zero_price):,}")
print(f"\nSample:")
print(zero_price.head(10).to_string())
print(f"\nStockCodes in zero-price rows:")
print(zero_price['StockCode'].value_counts().head(10).to_string())

Zero price rows remaining: 70

Sample:
      Invoice StockCode                      Description  Quantity         InvoiceDate  Price  Customer ID         Country  IsCancelled  TotalAmount
4674   489825     22076               6 RIBBONS EMPIRE          12 2009-12-02 13:34:00    0.0        16126  United Kingdom        False          0.0
6781   489998     48185              DOOR MAT FAIRY CAKE         2 2009-12-03 11:19:00    0.0        15658  United Kingdom        False          0.0
16107  490727         M                           Manual         1 2009-12-07 16:38:00    0.0        17231  United Kingdom        False          0.0
18738  490961     22065   CHRISTMAS PUDDING TRINKET POT          1 2009-12-08 15:25:00    0.0        14108  United Kingdom        False          0.0
18739  490961     22142     CHRISTMAS CRAFT WHITE FAIRY         12 2009-12-08 15:25:00    0.0        14108  United Kingdom        False          0.0
32916  492079     85042        ANTIQUE LILY FAIRY LIGHTS         8 

In [133]:
# Final data summary
print(f"{'='*55}")
print(f"CLEANED DATASET SUMMARY")
print(f"{'='*55}")
print(f"Total rows:        {len(df_clean):,}")
print(f"Normal orders:     {(~df_clean['IsCancelled']).sum():,}")
print(f"Cancelled orders:  {df_clean['IsCancelled'].sum():,}")
print(f"Unique customers:  {df_clean['Customer ID'].nunique():,}")
print(f"Unique invoices:   {df_clean['Invoice'].nunique():,}")
print(f"Unique products:   {df_clean['StockCode'].nunique():,}")
print(f"Countries:         {df_clean['Country'].nunique()}")
print(f"Date range:        {df_clean['InvoiceDate'].min().date()} to {df_clean['InvoiceDate'].max().date()}")
print(f"\nColumns: {list(df_clean.columns)}")
print(f"\nDtypes:")
print(df_clean.dtypes.to_string())
print(f"\nNull check:")
print(df_clean.isnull().sum().to_string())

CLEANED DATASET SUMMARY
Total rows:        797,885
Normal orders:     779,495
Cancelled orders:  18,390
Unique customers:  5,942
Unique invoices:   44,876
Unique products:   4,646
Countries:         41
Date range:        2009-12-01 to 2011-12-09

Columns: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'IsCancelled', 'TotalAmount']

Dtypes:
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID             Int64
Country                object
IsCancelled              bool
TotalAmount           float64

Null check:
Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
IsCancelled    0
TotalAmount    0
